In [14]:
import mysql.connector
import pandas as pd
import unicodedata
import numpy as np

# state

In [33]:
state = pd.read_parquet(r'..\datasets\processed\bd\1_states.parquet.gz')



state.to_parquet(r'..\datasets\processed\bd\1_states.parquet.gz', compression='gzip',index=False)

# Categories

In [11]:
categories = pd.read_parquet(r'..\datasets\processed\bd\2_categories.parquet.gz')

# Aplicar strip() a la columna 'name' para eliminar espacios en blanco adicionales
categories['name'] = categories['name'].str.strip()

# Convertir la columna 'name' a minúsculas para uniformizar los datos
categories['name'] = categories['name'].str.lower()

# Eliminar las filas duplicadas basadas en la columna 'name'
categories.drop_duplicates(subset=['name'], keep='first', inplace=True)

categories.to_parquet(r'..\datasets\processed\bd\2_categories.parquet.gz',compression='gzip', index=False)

# User yelp

In [13]:
user_yelp = pd.read_parquet(r'..\datasets\processed\bd\3_user_yelp.parquet.gz')
user_yelp['creation'] = user_yelp['creation'].dt.strftime('%Y-%m-%d %H:%M:%S')
user_yelp.to_parquet(r'..\datasets\processed\bd\3_user_yelp.parquet.gz',compression='gzip',index=False)

# User google

In [16]:
user_google = pd.read_parquet(r'..\datasets\processed\bd\4_user_google.parquet.gz')
user_google['creation'] = user_google['creation'].dt.strftime('%Y-%m-%d %H:%M:%S')
user_google.drop_duplicates(inplace=True)
user_google['user_id'] = user_google['user_id'].astype(str)
user_google['name'] = user_google['name'].replace(r'[^a-zA-Z\s]', '', regex=True)
user_google.drop(2023700,inplace=True)
user_google.dropna(inplace=True)
user_google = user_google.groupby(['user_id','name','creation','review_count'])['stars'].mean().reset_index()
user_google = user_google.drop_duplicates(subset=['user_id'], keep='first')


def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')

user_google.to_parquet(r'..\datasets\processed\bd\4_user_google.parquet.gz', index=False, compression='gzip')

# Business google

In [19]:
categories_google = pd.read_parquet(r'..\datasets\processed\bd\7_categories_google.parquet.gz')
business_google = pd.read_parquet(r'..\datasets\processed\bd\5_business_google.parquet.gz')


business_google.rename(columns={'state': 'state_id'}, inplace=True)
business_google["state_id"] = business_google["state_id"].replace("Florida", 1)
business_google["state_id"] = business_google["state_id"].replace("California", 2)
business_google["state_id"] = business_google["state_id"].replace("Illinois", 3)
business_google["state_id"] = business_google["state_id"].replace("New Jersey", 4)
business_google['name'] = business_google['name'].replace(r'[^a-zA-Z\s]', '', regex=True)
business_google.dropna(inplace=True)
business_google = business_google.drop_duplicates(subset=['gmap_id'], keep='first')
business_google.drop(22003,inplace=True)
business_google['avg_stars'] = business_google['avg_stars'].round(2)

business_google = business_google[business_google['gmap_id'].isin(categories_google['gmap_id'].unique())]


def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')


# Recorre todas las columnas del DataFrame
for column in business_google.columns:
    # Intenta corregir los caracteres no codificados como UTF-8 en cada celda
    business_google[column] = business_google[column].apply(lambda x: corregir_codificacion(str(x)) if isinstance(x, str) else x)


business_google.to_parquet(r'..\datasets\processed\bd\5_business_google.parquet.gz', compression='gzip',index=False)

# Business yelp

In [21]:
business_yelp = pd.read_parquet(r'..\datasets\processed\bd\6_business_yelp.parquet.gz')

business_yelp.rename(columns={'state': 'state_id'}, inplace=True)
business_yelp.rename(columns={'stars': 'avg_stars'}, inplace=True)
business_yelp["state_id"] = business_yelp["state_id"].replace("Florida", 1)
business_yelp["state_id"] = business_yelp["state_id"].replace("California", 2)
business_yelp["state_id"] = business_yelp["state_id"].replace("Illinois", 3)
business_yelp["state_id"] = business_yelp["state_id"].replace("New Jersey", 4)
business_yelp['name'] = business_yelp['name'].replace(r'[^a-zA-Z\s]', '', regex=True)
business_yelp.dropna(inplace=True)
business_yelp = business_yelp.drop_duplicates(subset=['business_id'], keep='first')
business_yelp['avg_stars'] = business_yelp['avg_stars'].round(2)

def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')


# Recorre todas las columnas del DataFrame
for column in business_yelp.columns:
    # Intenta corregir los caracteres no codificados como UTF-8 en cada celda
    business_yelp[column] = business_yelp[column].apply(lambda x: corregir_codificacion(str(x)) if isinstance(x, str) else x)

business_yelp.to_parquet(r'..\datasets\processed\bd\6_business_yelp.parquet.gz',compression='gzip',index=False)

# Categories google

In [23]:
categories_google = pd.read_parquet(r'..\datasets\processed\bd\7_categories_google.parquet.gz')

categories_google = categories_google[categories_google['categories_id'].isin(categories['categories_id'])]

categories_google = categories_google[categories_google['gmap_id'].isin(business_google['gmap_id'])]

def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')


# Recorre todas las columnas del DataFrame
for column in categories_google.columns:
    # Intenta corregir los caracteres no codificados como UTF-8 en cada celda
    categories_google[column] = categories_google[column].apply(lambda x: corregir_codificacion(str(x)) if isinstance(x, str) else x)


categories_google.to_parquet(r'..\datasets\processed\bd\7_categories_google.parquet.gz',compression='gzip',index=False)

# Categories yelp

In [26]:
categories_yelp = pd.read_parquet(r'..\datasets\processed\bd\8_categories_yelp.parquet.gz')

categories_yelp = categories_yelp[categories_yelp['categories_id'].isin(categories['categories_id'])]

categories_yelp = categories_yelp[categories_yelp['business_id'].isin(business_yelp['business_id'])]

def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')


# Recorre todas las columnas del DataFrame
for column in categories_yelp.columns:
    # Intenta corregir los caracteres no codificados como UTF-8 en cada celda
    categories_yelp[column] = categories_yelp[column].apply(lambda x: corregir_codificacion(str(x)) if isinstance(x, str) else x)


categories_yelp.to_parquet(r'..\datasets\processed\bd\8_categories_yelp.parquet.gz', compression='gzip',index=False)

# Reviews google

In [28]:
reviews_google = pd.read_parquet(r'..\datasets\processed\bd\9_reviews_google.parquet.gz')

reviews_google['date'] = reviews_google['date'].dt.strftime('%Y-%m-%d %H:%M:%S').str.replace('\r', '')
reviews_google['resp_date'] = reviews_google['resp_date'].dt.strftime('%Y-%m-%d %H:%M:%S').str.replace('\r', '')

reviews_google['user_id'] = reviews_google['user_id'].astype(str)

categories_yelp = categories_yelp[categories_yelp['categories_id'].isin(categories['categories_id'])]

reviews_google.loc[reviews_google['resp_date'].isna(), 'resp_date'] = reviews_google.loc[reviews_google['resp_date'].isna(), 'date']


def round_resp_sentiment(value):
    if value < -0.5:
        return 0
    elif -0.5 <= value < 0.5:
        return 1
    else:
        return 2


reviews_google['resp_sentiment'] = reviews_google['resp_sentiment'].apply(round_resp_sentiment)
reviews_google['resp_sentiment'] = reviews_google['resp_sentiment'].astype(str)

reviews_google.to_parquet(r'..\datasets\processed\bd\9_reviews_google.parquet.gz', compression='gzip', index=False)

# Reviews yelp

In [30]:
reviews_yelp = pd.read_parquet(r'..\datasets\processed\bd\10_reviews_yelp.parquet.gz')

reviews_yelp['date'] = reviews_yelp['date'].dt.strftime('%Y-%m-%d %H:%M:%S').str.replace('\r', '')

reviews_yelp = reviews_yelp[reviews_yelp['business_id'].isin(business_yelp['business_id'])]

reviews_yelp = reviews_yelp[reviews_yelp['user_id'].isin(user_yelp['user_id'])]

reviews_yelp.to_parquet(r'..\datasets\processed\bd\10_reviews_yelp.parquet.gz',compression='gzip',index=False)